<a href="https://colab.research.google.com/github/ardevr/MVA-MP3/blob/master/DQN_project_MVA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
!pip install scikit-video

    100% |████████████████████████████████| 2.3MB 13.9MB/s 


In [2]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json


from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization,Flatten

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The act function is used to select the next action taken by the agent. However, as the rewards obtained by the agent are correlated in time, always taking the action that maximizes the total reward until now may be suboptimal. The parameter $\varepsilon$ insures that the agent still explores possible actions throughout the training and does not simply select the action with the highest estimated reward which can be suboptimal in reality.

In short, $\varepsilon$ is essential for exploration of the action space.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
  
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=100 # set small when debugging
epochs_test=20 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The array ```board``` represents the different malus and bonus on the discretized island. When the mouse visit a cell, it eats the bonus or malus and cell's value is set to $0$.

The array  ```position```  is used to track the position of the mouse on the board, as after each action the previous position of the mouse will be marked by a $1$ and all other cells will be at $0$.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        

    def learned_act(self, s):
        return np.random.randint(0,self.n_action)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
      
      state = env.reset()
      game_over = False
      win = 0
      lose = 0
      
      while not game_over:
        
        # Play while the agent has not encountered a terminal state
        action = agent.learned_act(state)
        prev_state = state
        state, reward, game_over = env.act(action)

        if reward > 0:
          win = win + reward
        if reward < 0:
          lose = lose - reward
          
      # Save as a mp4
      env.draw(prefix+str(e))

      # Update stats
      score = score + win-lose

      print("Win/lose count {}/{}. Average score ({})".format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [11]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random19.mp4'))

Win/lose count 9.0/11.0. Average score (-2.0)
Win/lose count 14.5/15.0. Average score (-1.25)
Win/lose count 12.0/16.0. Average score (-2.1666666666666665)
Win/lose count 14.0/11.0. Average score (-0.875)
Win/lose count 8.5/9.0. Average score (-0.8)
Win/lose count 8.5/12.0. Average score (-1.25)
Win/lose count 9.5/18.0. Average score (-2.2857142857142856)
Win/lose count 11.0/15.0. Average score (-2.5)
Win/lose count 11.5/18.0. Average score (-2.9444444444444446)
Win/lose count 9.0/10.0. Average score (-2.75)
Win/lose count 5.5/5.0. Average score (-2.4545454545454546)
Win/lose count 10.0/11.0. Average score (-2.3333333333333335)
Win/lose count 7.5/17.0. Average score (-2.8846153846153846)
Win/lose count 9.0/11.0. Average score (-2.8214285714285716)
Win/lose count 12.0/18.0. Average score (-3.033333333333333)
Win/lose count 10.0/12.0. Average score (-2.96875)
Win/lose count 7.0/11.0. Average score (-3.0294117647058822)
Win/lose count 10.0/16.0. Average score (-3.1944444444444446)
Win/los

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




For a policy $\pi$, a state $s$ and an action $a$  :

\begin{align*}
Q^{\pi}(s,a) &= E_{p^{\pi}}\left[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a\right]  \\
&= r(s,a) + \gamma \sum_{s'}E_{p^{\pi}}\left[\sum_{t= 1}^{T}\gamma^{t-1}r(s_{t},a_{t})\mathbb{1}_{\{s_{1} = s'\}}|s_{0}=s,a_{0}=a\right] \\
&=  r(s,a) + \gamma \sum_{s'}\mathbb{P}(s'|s,a)E_{p^{\pi}}\left[\sum_{t= 1}^{T}\gamma^{t-1}r(s_{t},a_{t}) | s_{1}=s'\right] \\
 & = r(s,a) + \gamma \sum_{s',a'}\mathbb{P}(s'|s,a)\pi(a'|s')E_{p^{\pi}}\left[\sum_{t= 1}^{T}\gamma^{t-1}r(s_{t},a_{t}) | s_{1}=s',a_{1}=a'\right] \\
&=  r(s,a) + \gamma \mathbb{E}_{(s',a')\sim (\mathbb{P}(.|s,a),\pi(.|s'))}(Q^{\pi}(s',a')) \\
\end{align*}

where $\mathbb{P}(.|s,a)$ is the model probability when taking action $a$ in state $s$ and all equalities are justified by the fact that a sufficient statistics for the next state $s_{t+1}$ is the current state $s_{t}$ and the action $a_{t}$. 

For a policy $\pi$, let's note $\pi = (a_{0},\pi')$ (i.e the first action is $a_{0}$ and then the action are distributed according to $\pi'$) then the optimal $Q$-function satisifes :

\begin{align*}
Q^{\star}(s,a) &= \max_{\pi} Q^{\pi}(s,a) \\
&= r(s,a)  + \gamma\max_{a_{0},\pi'} \mathbb{E}_{(s',a')\sim (\mathbb{P}(.|s,a),\pi(.|s'))}(Q^{\pi}(s',a')) \\
&= r(s,a)  + \gamma\max_{a_{0}} \mathbb{E}_{s'\sim \mathbb{P}(.|s,a)}(Q^{\star}(s',a_{0})) \\
&= r(s,a)  + \gamma\mathbb{E}_{s'\sim \mathbb{P}(.|s,a)}(\max_{a_{0}} Q^{\star}(s',a_{0})) \\
\end{align*}

In that case the model state distribution is deterministic so the objective $\mathcal{L}$ is such that $\mathcal{L}(\theta) = || r(s,a) + \gamma \max_{a'} Q(s',a',\theta) - Q(s,a,\theta)||^{2} $
where $s'$ is the state of the agent after taking action $a$ in state $s$. It is a sensible objective because it means we are fitting a network to represent a function that satisfy the equality characterising the optimal $Q$-function. Thus, as there is a unique function that satisfy this equality (as this can be seen as a fix point of a contraction operator) is the network converges it will be toward the optimal $Q$-function. Finally, extracting the optimal policy is not diffuclt as $\pi^{\star}(.|s) = \arg\max_{a} Q^{\star}(s,a)$.




***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
      self.max_memory = max_memory
      self.memory = list()

    def remember(self, m):
      len_memory = len(self.memory)
      #Check is the buffer is full
      if len_memory >= self.max_memory :
        #if full delete the first element
        self.memory = self.memory[1::]
      self.memory.append(m)
        
    def random_access(self):
      #Draws a uniform random index of the memory list
      random_index = np.random.randint(0,len(self.memory))
      return self.memory[random_index]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 1 == 0:
            env.draw(prefix+str(e+1))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {:.2f}/{:.2f} ({:.2f})"
              .format(e+1, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
      #Returns an action based on the greedy policy with respect to the learned Q-function
        predicted_q = (self.model.predict(s[None,:]))[0]
        max_action = np.argmax(predicted_q)
        return max_action

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            
            s, n_s, a, r, game_over = self.memory.random_access()
            target_q[i] = self.model.predict(s[None,:])
            predicted_q = self.model.predict(n_s[None,:])[0]
            
            # Replace the value of the q-function only for the action played
            # and set the value of the other actions to the value predicted by the network at the previous
          
            if game_over:
                
                target_q[i][a] = r
                
            else:
                
                target_q[i][a] = r + self.discount*max(predicted_q)
            
            
            input_states[i,:] = s 
                
        
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -2, 2)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
                
        # NN Model
       
        model = Sequential()
        model.add(Flatten(input_shape = (5,5,self.n_state)))
        model.add(Dense(32,activation  = 'relu'))
        model.add(Dense(4,activation = 'linear'))
        
        model.compile(sgd(lr=lr, decay=1e-7, momentum=0), "mse")
        self.model = model
        

In [15]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.11, epsilon = 0.1, memory_size=200, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train90.mp4'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 001/100 | Loss 0.0269 | Win/lose count 2.00/4.00 (-2.00)
Epoch 002/100 | Loss 0.0163 | Win/lose count 3.00/1.00 (2.00)
Epoch 003/100 | Loss 0.0035 | Win/lose count 7.50/8.00 (-0.50)
Epoch 004/100 | Loss 0.0121 | Win/lose count 4.00/3.00 (1.00)
Epoch 005/100 | Loss 0.0007 | Win/lose count 2.50/2.00 (0.50)
Epoch 006/100 | Loss 0.0038 | Win/lose count 6.00/9.00 (-3.00)
Epoch 007/100 | Loss 0.0076 | Win/lose count 3.50/6.00 (-2.50)
Epoch 008/100 | Loss 0.0047 | Win/lose count 5.50/6.00 (-0.50)
Epoch 009/100 | Loss 0.0042 | Win/lose count 4.00/3.00 (1.00)
Epoch 010/100 | Loss 0.0011 | Win/lose count 2.00/1.00 (1.00)
Epoch 011/100 | Loss 0.0033 | Win/lose count 13.00/4.00 (9.00)
Epoch 012/100 | Loss 0.0003 | Win/lose count 3.50/0.00 (3.50)
Epoch 013/100 | Loss 0.0047 | Win/lose count 4.50/2.00 (2.50)
Epoch 014/100 | Loss 0.0132 | Win/lose count 4.00/4.00 (0.00)
Epoch 

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(32,kernel_size = (2,2), activation = 'relu',input_shape = (5,5,self.n_state)))
        model.add(Conv2D(16,kernel_size = (2,2), activation = 'relu'))
        model.add(Flatten())
        model.add(Dense(4,activation = 'linear'))
        
        model.compile(sgd(lr=lr, decay=1e-7, momentum=0), "mse")
        self.model = model

In [17]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.11, epsilon = 0.1, memory_size=30, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train99.mp4'))

Epoch 001/100 | Loss 0.0005 | Win/lose count 2.00/1.00 (1.00)
Epoch 002/100 | Loss 0.0038 | Win/lose count 7.50/9.00 (-1.50)
Epoch 003/100 | Loss 0.0026 | Win/lose count 3.50/3.00 (0.50)
Epoch 004/100 | Loss 0.0033 | Win/lose count 3.50/1.00 (2.50)
Epoch 005/100 | Loss 0.0164 | Win/lose count 3.00/3.00 (0.00)
Epoch 006/100 | Loss 0.0172 | Win/lose count 5.00/3.00 (2.00)
Epoch 007/100 | Loss 0.0000 | Win/lose count 1.50/2.00 (-0.50)
Epoch 008/100 | Loss 0.0215 | Win/lose count 2.50/2.00 (0.50)
Epoch 009/100 | Loss 0.0245 | Win/lose count 7.00/3.00 (4.00)
Epoch 010/100 | Loss 0.0018 | Win/lose count 7.00/5.00 (2.00)
Epoch 011/100 | Loss 0.0482 | Win/lose count 12.00/3.00 (9.00)
Epoch 012/100 | Loss 0.0234 | Win/lose count 8.00/4.00 (4.00)
Epoch 013/100 | Loss 0.0027 | Win/lose count 5.50/3.00 (2.50)
Epoch 014/100 | Loss 0.0259 | Win/lose count 8.00/2.00 (6.00)
Epoch 015/100 | Loss 0.0454 | Win/lose count 15.00/4.00 (11.00)
Epoch 016/100 | Loss 0.0005 | Win/lose count 1.00/2.00 (-1.00)
Ep

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [30]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.09, epsilon = 0.1, memory_size=150, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.11, epsilon = 0.1, memory_size=150, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test_temp_0.3_')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test_temp_0.3_')

Test of the CNN
Win/lose count 9.5/0. Average score (9.5)
Win/lose count 7.5/0. Average score (8.5)
Win/lose count 5.5/0. Average score (7.5)
Win/lose count 7.0/0. Average score (7.375)
Win/lose count 15.5/0. Average score (9.0)
Win/lose count 4.0/0. Average score (8.166666666666666)
Win/lose count 12.5/0. Average score (8.785714285714286)
Win/lose count 9.0/1.0. Average score (8.6875)
Win/lose count 7.5/0. Average score (8.555555555555555)
Win/lose count 6.0/1.0. Average score (8.2)
Win/lose count 2.5/0. Average score (7.681818181818182)
Win/lose count 9.5/0. Average score (7.833333333333333)
Win/lose count 10.5/0. Average score (8.038461538461538)
Win/lose count 10.5/0. Average score (8.214285714285714)
Win/lose count 2.5/0. Average score (7.833333333333333)
Win/lose count 4.5/0. Average score (7.625)
Win/lose count 7.5/0. Average score (7.617647058823529)
Win/lose count 5.5/0. Average score (7.5)
Win/lose count 8.5/0. Average score (7.552631578947368)
Win/lose count 1.5/0. Average s

In [31]:
HTML(display_videos('cnn_test_temp_0.3_18.mp4'))

In [32]:
HTML(display_videos('fc_test_temp_0.3_19.mp4'))

The first observations is that the fully connected network overfits greatly but the convolutionnal network exhibits strong performance. However, in both cases the agent seems to be blocked in a local optima and is keep taking action with sub-optimal rewards. This may be caused by a lack of exploration in training where not enough state-action pairs are visited by the agent. 

When the temperature is increased the average reward goes up, as it should be because the number of states with a positive reward increase. However the exploration problem is still present.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,prefix=''):
    score = 0
    loss = 0

    for e in range(epoch):
      
        state = env.reset()
        game_over = False

        win = 0
        lose = 0
        t = 0 
        
        # epsilon decreases linearly troughout the game
        epsilon = np.linspace(agent.epsilon/100,agent.epsilon, env.max_time+1)[::-1]

        while not game_over:
            
            agent.set_epsilon(epsilon[t])
            action = agent.act(state,train = True)
            prev_state = state
            state, reward, game_over = env.act(action,train = True)

            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
            
            t +=1
            
        if e % 1 == 0:
            env.draw(prefix+str(e+1))

        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {:.2f}/{:.2f} ({:.2f})"
              .format(e+1, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
    print('Average Score = ', score/epoch)

In [0]:
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.maalus_postion = 0*self.board

        self.x = 0
        self.y = 1

        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action,train = False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = 0
        if train:
          reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1
        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus
        self.malus_position = 0*self.board

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
       
        
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state    

In [35]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.09, epsilon = 1, memory_size=100, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore100.mp4'))

Epoch 001/100 | Loss 0.0114 | Win/lose count 12.00/26.90 (-14.90)
Epoch 002/100 | Loss 0.0023 | Win/lose count 5.00/19.00 (-14.00)
Epoch 003/100 | Loss 0.0015 | Win/lose count 6.00/20.10 (-14.10)
Epoch 004/100 | Loss 0.0032 | Win/lose count 6.50/21.20 (-14.70)
Epoch 005/100 | Loss 0.0092 | Win/lose count 8.00/18.40 (-10.40)
Epoch 006/100 | Loss 0.0081 | Win/lose count 7.00/20.70 (-13.70)
Epoch 007/100 | Loss 0.0019 | Win/lose count 14.00/17.40 (-3.40)
Epoch 008/100 | Loss 0.0168 | Win/lose count 10.00/19.90 (-9.90)
Epoch 009/100 | Loss 0.0049 | Win/lose count 6.50/18.20 (-11.70)
Epoch 010/100 | Loss 0.0115 | Win/lose count 10.50/18.70 (-8.20)
Epoch 011/100 | Loss 0.0059 | Win/lose count 17.00/13.50 (3.50)
Epoch 012/100 | Loss 0.0018 | Win/lose count 5.50/18.00 (-12.50)
Epoch 013/100 | Loss 0.0210 | Win/lose count 17.00/14.40 (2.60)
Epoch 014/100 | Loss 0.0123 | Win/lose count 14.50/12.80 (1.70)
Epoch 015/100 | Loss 0.0030 | Win/lose count 13.00/16.90 (-3.90)
Epoch 016/100 | Loss 0.0036

In [36]:
HTML(display_videos('cnn_train_explore97.mp4'))

In [37]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore19.mp4'))

Win/lose count 27.5/5.0. Average score (22.5)
Win/lose count 3.5/2.0. Average score (12.0)
Win/lose count 25.0/4.0. Average score (15.0)
Win/lose count 26.0/4.0. Average score (16.75)
Win/lose count 22.0/5.0. Average score (16.8)
Win/lose count 26.0/5.0. Average score (17.5)
Win/lose count 13.0/1.0. Average score (16.714285714285715)
Win/lose count 20.5/3.0. Average score (16.8125)
Win/lose count 22.0/8.0. Average score (16.5)
Win/lose count 23.5/3.0. Average score (16.9)
Win/lose count 12.0/3.0. Average score (16.181818181818183)
Win/lose count 21.5/1.0. Average score (16.541666666666668)
Win/lose count 19.0/2.0. Average score (16.576923076923077)
Win/lose count 19.0/2.0. Average score (16.607142857142858)
Win/lose count 16.0/1.0. Average score (16.5)
Win/lose count 15.0/3.0. Average score (16.21875)
Win/lose count 23.5/1.0. Average score (16.58823529411765)
Win/lose count 24.5/4.0. Average score (16.805555555555557)
Win/lose count 18.0/0. Average score (16.86842105263158)
Win/lose co

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***